In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

# Any results you write to the current directory are saved as output.

In [2]:
data1 = np.load('train-1.npy')
data2 = np.load('train-2.npy')
data3 = np.load('train-3.npy')
data4 = np.load('train-4.npy')
test = np.load('test.npy')

In [3]:
from skimage.transform import resize
from tqdm import tqdm

for i in tqdm(range(data1.shape[0])):
    data1[i][0] = resize(data1[i][0], (28, 28))
for i in tqdm(range(data2.shape[0])):
    data2[i][0] = resize(data2[i][0], (28, 28))
for i in tqdm(range(data3.shape[0])):
    data3[i][0] = resize(data3[i][0], (28, 28))
for i in tqdm(range(data4.shape[0])):
    data4[i][0] = resize(data4[i][0], (28, 28))
data1[:, 0] /= 255
data2[:, 0] /= 255
data3[:, 0] /= 255
data4[:, 0] /= 255


  0%|          | 0/83246 [00:00<?, ?it/s]/home/raccooncoder/anaconda3/envs/pytorch/lib/python3.7/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/raccooncoder/anaconda3/envs/pytorch/lib/python3.7/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
100%|██████████| 83247/83247 [00:40<00:00, 2051.76it/s]


In [4]:
for i in tqdm(range(test.shape[0])):
    test[i] = resize(test[i], (28, 28))
test /= 255

print(test.shape[0])


100%|██████████| 83247/83247 [00:42<00:00, 1959.56it/s]


83247


In [5]:
label_to_num = {}
num_to_label = {}
c = 0
for i in range(len(data1)):
    l = len(label_to_num)
    label_to_num.setdefault(data1[i][1], c)
    if len(label_to_num) > l:
        num_to_label.setdefault(c, data1[i][1])
        c += 1
        
print(len(label_to_num))

1000


In [6]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [32]:
#hyperparameters
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
num_epochs = 5
num_classes = 1000
batch_size = 1024
learning_rate = 0.001

In [23]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(128),
            nn.ReLU())
        self.fc = nn.Linear(7*7*128, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [18]:
import torch.utils.data as utils

train = np.concatenate((data1, data2, data3), axis=0)
train_features = train[:, 0]
train_labels = train[:, 1]
for i in range(len(train_labels)):
    train_labels[i] = label_to_num[train_labels[i]]

valid_features = data4[:, 0]
valid_labels = data4[:, 1]
for i in range(len(valid_labels)):
    valid_labels[i] = label_to_num[valid_labels[i]]

tensor_train_features = torch.stack([torch.unsqueeze(torch.Tensor(i), 0) for i in train_features])
tensor_train_labels = torch.Tensor(train_labels.astype('int32')).type(torch.LongTensor)
tensor_valid_features = torch.stack([torch.unsqueeze(torch.Tensor(i), 0) for i in valid_features])
tensor_valid_labels = torch.Tensor(valid_labels.astype('int32')).type(torch.LongTensor)

train_dataset = utils.TensorDataset(tensor_train_features, tensor_train_labels)
valid_dataset = utils.TensorDataset(tensor_valid_features, tensor_valid_labels)

KeyError: 973

In [19]:
train_loader = utils.DataLoader(dataset=train_dataset,
                                batch_size=batch_size, 
                                shuffle=True)
valid_loader = utils.DataLoader(dataset=valid_dataset,
                                batch_size=batch_size,
                                shuffle=False)

In [34]:
import torch.nn.functional as F
import torchvision.models as models

model = models.resnet18(pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_step = len(train_loader)

for epoch in range(num_epochs):
    model.train()
    
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    
    model.eval()
    loss = 0
    correct = 0
    
    with torch.no_grad():
        for i, (images, labels) in enumerate(valid_loader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            
            loss += F.cross_entropy(outputs, labels, reduction = 'sum').item()

            pred = outputs.data.max(1, keepdim=True)[1]
            correct += pred.eq(labels.data.view_as(pred)).cpu().sum()

    loss /= len(valid_loader.dataset)
        
    print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        loss, correct, len(valid_loader.dataset),
        100. * correct / len(valid_loader.dataset)))



Epoch [1/5], Step [100/244], Loss: 6.3811
Epoch [1/5], Step [200/244], Loss: 2.3208

Average loss: 2.4674, Accuracy: 34305/83247 (41.000%)

Epoch [2/5], Step [100/244], Loss: 0.7242
Epoch [2/5], Step [200/244], Loss: 0.4306

Average loss: 0.7662, Accuracy: 65926/83247 (79.000%)

Epoch [3/5], Step [100/244], Loss: 0.2815
Epoch [3/5], Step [200/244], Loss: 0.2265

Average loss: 0.4932, Accuracy: 72231/83247 (86.000%)

Epoch [4/5], Step [100/244], Loss: 0.1097
Epoch [4/5], Step [200/244], Loss: 0.1562

Average loss: 0.6080, Accuracy: 70257/83247 (84.000%)

Epoch [5/5], Step [100/244], Loss: 0.0795
Epoch [5/5], Step [200/244], Loss: 0.0790

Average loss: 0.4311, Accuracy: 74163/83247 (89.000%)



In [ ]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in valid_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the govno test images: {} %'.format(100 * correct / total))


In [ ]:
y = [0 for i in range(len(test))]
data_x = torch.stack([torch.from_numpy(i).type(torch.FloatTensor) for i in test])
